In [5]:
def u3D(r,z,E=100e6,nu=0.3,F=100e3):
    Rad = np.sqrt(r**2 + z**2)
    G = E/(2*(1+nu))
    fak = F/(4*np.pi*G*Rad)
    return fak * (2*(1-nu) + (z/Rad)**2)

In [6]:
def u1D(r,z,E=100e6,nu=0.3,F=100e3):
    Rad = np.sqrt(r**2 + z**2)
    Es = E*(1-nu)/((1+nu)*(1-2*nu))
    fak = F/(2*np.pi*Es*Rad)
    return fak * (2 + (z/Rad)**2)

In [ ]:
def u3Dsym(x,z,E=100e6,nu=0.3,F=100e3):
    r = np.abs(x)
    Rad = np.sqrt(r**2 + z**2)
    G = E/(2*(1+nu))
    fak = F/(4*np.pi*G*Rad)
    return fak * (2*(1-nu) + (z/Rad)**2)

In [ ]:
def interactive_point_settlement():
    @interact(#c=widgets.BoundedFloatText(value=0.3,min=0, description='$c$ / MPa'),
               E=widgets.BoundedFloatText(value=100, min=1e-3, max=1e3, description='$E$ / MPa'),
               F=widgets.BoundedFloatText(value=100, min=1, max=1e4, description='$F_z$ / kN'),
               nu=widgets.BoundedFloatText(value=0.3, min=0, max=0.49, description='$\\nu$')
    )
    def plot_MC_states(E,F,nu):
        E *= 1e6
        F *= 1e3
        z = np.array([0.1,0.5,1,2,3,5]) # m
        r = np.linspace(0,5,100)
        sns.set_palette("Paired",n_colors=12)
        fig, ax = plt.subplots(figsize=(18,8),ncols=3)
        for i in z:
            ax[0].plot(r,-u3D(r,i,E,nu,F)*1e3-i,label="$z = %.2f$ m" %i)
            ax[0].plot(r,-u1D(r,i,E,nu,F)*1e3-i,label="$z = %.2f$ m" %i,ls='--')
            ax[0].axhline(-i,ls=':',color='black')

        ax[0].set_xlabel('$r$ / m')
        ax[0].set_ylabel('$u_z$ / mm + $z$ / m')
        ax[0].set_yticks([0,-1,-2,-3,-4,-5])
        ax[0].set_yticklabels([0,1,2,3,4,5])
        ax[0].set_xlim(0,5)
        ax[0].spines['bottom'].set_position(('data', 0))
        ax[0].xaxis.set_label_position('top') 
        ax[0].xaxis.tick_top()
        #
        z = np.linspace(0.1,5,100)
        ax[1].plot(z,u3D(0,z,E,nu,F)*1e3,label='3D solution')
        ax[1].plot(z,u1D(0,z,E,nu,F)*1e3,label='$\\epsilon_{xx}=\\epsilon_{yy} = 0$',ls='--')
        ax[1].set_xlabel('$z$ / m')
        ax[1].set_ylabel('$u_z(r=0)$ / mm')
        ax[1].set_xlim(left=0)
        ax[1].set_ylim(bottom=0)
        ax[1].legend()
        #
        r = np.linspace(0.1,5,100)
        ax[2].plot(z,u3D(r,0,E,nu,F)*1e3,label='3D solution')
        ax[2].plot(z,u1D(r,0,E,nu,F)*1e3,label='$\\epsilon_{xx}=\\epsilon_{yy} = 0$',ls='--')
        ax[2].set_xlabel('$r$ / m')
        ax[2].set_ylabel('$u_z(z=0) = s(r)$ / mm')
        ax[2].set_xlim(left=0)
        ax[2].set_ylim(bottom=0)
        ax[2].legend()
        fig.tight_layout();

In [ ]:
def plot_settlement_simple(E=1,F=1,nu=0.3):
    E *= 1e6
    F *= 1e3
    z = np.array([0.1,0.5,1,2,3,5]) # m
    r = np.linspace(0,5,100)
    fig, ax = plt.subplots()
    for i in z:
        ax.plot(r,-u3D(r,i,E,nu,F)*1e3-i,label="$z = %.2f$ m" %i)
        ax.axhline(-i,ls=':',color='black')

    ax.set_xlabel('$r$ / m')
    ax.set_ylabel('$u_z$ / mm + $z$ / m')
    ax.set_yticks([0,-1,-2,-3,-4,-5])
    ax.set_yticklabels([0,1,2,3,4,5])
    ax.set_xlim(0,5)
    ax.spines['bottom'].set_position(('data', 0))
    ax.xaxis.set_label_position('top') 
    ax.xaxis.tick_top()
    fig.tight_layout();

In [ ]:
from ipywidgets import widgets
from ipywidgets import interact
def interactive_superposition_u():
    @interact(n_loads=widgets.BoundedIntText(value=3, min=1., max=10, step=1., description='number of loads', style={'description_width': 'initial'})
    )
    def plot_isobars_zz_Boussinesq_n(n_loads):
        nu = 0.3
        fig, ax = plt.subplots(figsize=(9,9))
        xs = np.linspace(-6,6,200)
        zs = np.linspace(0.5,5,100)
        uz = np.zeros((len(xs),len(zs)))
        for i,x in enumerate(xs):
            for j,z in enumerate(zs):
                for x0 in (np.linspace(0,n_loads,n_loads)-n_loads/2):
                    uz[i,j] += u3Dsym(x-x0,z,1,0.3,1)
        
        u_tot =xs*0
        for x0 in (np.linspace(0,n_loads,n_loads)-n_loads/2):
            ui = u3Dsym(xs-x0,0.5,1,0.3,1)
            u_tot += ui
            ax.plot(xs,ui,color='black',alpha=0.3)
            ax.annotate('',xy=(x0,0),xytext=(x0,0.5),arrowprops=dict(facecolor='black'))
        ax.plot(xs,u_tot,color='red')
        X, Z = np.meshgrid(xs,zs)
        cp = plt.contour(X, -Z, uz.T,levels=20)
        ax.clabel(cp, inline=True, 
                  fontsize=14)
        ax.set_xlabel('$x$ / m')
        ax.set_ylabel('$z$ / m')
        ax.axhline(0)
        axlim = np.max(u_tot)+0.05 #2.2 fuer plot
        ax.set_ylim(-5,axlim)
        ax.set_yticks([-1,-2,-3,-4,-5])
        ax.set_yticklabels([1,2,3,4,5])
        ax.spines['bottom'].set_position(('data', axlim))
        ax.xaxis.set_label_position('top') 
        ax.xaxis.tick_top()
        ax.set(adjustable='box', aspect='equal')
        fig.tight_layout();
        #fig.savefig('superposition_%i.pdf' %n_loads);

In [ ]:
def u3D_Rechteck(t,a,b,E=100e6,nu=0.3,s0=100.e3):
    Rad = np.sqrt(a**2+b**2+t**2)
    fak1 = s0*(1-nu**2)/(np.pi * E)
    fak2 = s0*(1-nu-2*nu**2)/(2*np.pi * E)
    t1 = a*np.log( (b + np.sqrt(a**2+b**2)) * np.sqrt(a**2+t**2) / (a*(b+Rad)) )
    t2 = b*np.log( (a + np.sqrt(a**2+b**2)) * np.sqrt(b**2+t**2) / (b*(a+Rad)) )
    t3 = t * np.arctan(a*b/(t*Rad))
    return fak1 * (t1 + t2) + fak2 * t3

In [ ]:
def u3D_Kreis(t,R,E,nu,s0):
    wrzl = np.sqrt(t**2 + R**2)
    t1 = (1-nu**2)*(R+t-wrzl)
    t2 = t*(1+nu)/2 * (1-t/wrzl)
    return 2*s0/E * (t1-t2)

In [ ]:
def interactive_rect_settlement():
    @interact(#c=widgets.BoundedFloatText(value=0.3,min=0, description='$c$ / MPa'),
               E=widgets.BoundedFloatText(value=100, min=1e-3, max=1e3, description='$E$ / MPa'),
               s0=widgets.BoundedFloatText(value=100, min=1, max=1e4, description='$q_0$ / kPa'),
               a=widgets.BoundedFloatText(value=3, min=0.1, max=10, description='$a$ / m'),
               b=widgets.BoundedFloatText(value=2, min=0.1, max=10, description='$b$ / m')
    )
    def plot_MC_states(E,s0,a,b):
        E *= 1e6
        s0 *=1e3
        t = np.linspace(0.1,100,1000) # m
        sns.set_palette("rocket")
        fig, ax = plt.subplots(figsize=(12,8))
        for nu in [0.0,0.1,0.2,0.3,0.4,0.5]:
            ax.plot(t,u3D_Rechteck(t,a,b,E,nu,s0)*1e3,label='$\\nu = %.1f$' %nu)
        ax.set_xlabel('$t$ / m')
        ax.set_ylabel('$s$ / mm')
        ax.set_xlim(left=0)
        ax.set_ylim(bottom=0)
        ax.legend()
        fig.tight_layout();

In [1]:
def interactive_rect_circ_settlement():
    @interact(E=widgets.BoundedFloatText(value=100, min=1e-3, max=1e3, description='$E$ / MPa'),
              s0=widgets.BoundedFloatText(value=100, min=1, max=1e4, description='$q_0$ / kPa'),
              a=widgets.BoundedFloatText(value=3, min=0.1, max=10, description='$a$ / m'),
              b=widgets.BoundedFloatText(value=2, min=0.1, max=10, description='$b$ / m')
    )
    def plot_MC_states(E,s0,a,b):
        E *= 1e6
        s0 *=1e3
        R = np.sqrt(a*b/np.pi)
        t = np.linspace(0.1,100,1000) # m
        colors = sns.color_palette("rocket").as_hex()
        fig, ax = plt.subplots(figsize=(12,8))
        for i,nu in enumerate([0.15,0.3,0.45]):
            uMR = 4*u3D_Rechteck(t,a/2,b/2,E,nu,s0)*1e3
            uER = u3D_Rechteck(t,a,b,E,nu,s0)*1e3
            uMK = u3D_Kreis(t,R,E,nu,s0)*1e3
            ax.plot(t,uMR,label='$\\nu = %.1f$' %nu,color=colors[i])
            ax.plot(t,uER,color=colors[i],ls=':')
            ax.plot(t,uMK,color=colors[i],ls='--')
        ax.set_xlabel('$t$ / m')
        ax.set_ylabel('$s$ / mm')
        ax.plot(0,0,color='black',ls='-',label='Rechteck, Mitte')
        ax.plot(0,0,color='black',ls=':',label='Rechteck, Eckpunkt')
        ax.plot(0,0,color='black',ls='--',label='Kreis mit R = %.3f m, Mitte' %R)
        ax.set_xlim(left=0)
        ax.set_ylim(bottom=0)
        ax.legend(ncol=2)
        fig.tight_layout();

In [ ]:
def i_Rechteck(z,a,b):
    R = np.sqrt(a**2 + b**2 + z**2)
    term1 = b*z/(b**2+z**2) * a*(R**2 + z**2)/((a**2+z**2)*R)
    term2 = np.arctan( b/z * (a*(a**2+b**2) - 2*a*z * (R-z))/((a**2+b**2)*(R-z) - z*(R-z)**2) )
    return (term1+term2)/(2*np.pi)

In [ ]:
#identische Funktion, aber funktioniert mit anderen array strukturen
def sigma_z_rectangle2(x,y,z,s0,a,b):
    xi = np.array([-a/2,-a/2,a/2,a/2])
    yi = np.array([-b/2,b/2,b/2,-b/2])
    sigmai = 0.
    for i in range(0,4):
        Ri = np.sqrt( (x+xi[i])**2 + (y+yi[i])**2 + z**2 )    
        term1i = ( 1/((x+xi[i])**2 + z**2) + 1/((y+yi[i])**2 + z**2) ) * ( (x+xi[i])*(y+yi[i])*z ) / Ri
        term2i = np.arctan( ( (x+xi[i])*(y+yi[i]) ) / (Ri * z) )
        signi = (-1)**i    
        sigmai += signi*(term1i + term2i)
    return s0/(2 * np.pi) * sigmai

In [ ]:
def u3D_inf(a,b,E=100e6,nu=0.3,s0=100.e3):
    fak1 = s0*(1-nu**2)/(np.pi * E)
    t1 = a*np.log( (b + np.sqrt(a**2+b**2))/ a )
    t2 = b*np.log( (a + np.sqrt(a**2+b**2))/ b )
    return fak1 * (t1 + t2)

In [2]:
import scipy as scp
def interactive_grenztiefe():
    @interact(a=widgets.BoundedFloatText(value=3, min=0.1, max=10, description='$a$ / m'),
              b=widgets.BoundedFloatText(value=2, min=0.1, max=10, description='$b$ / m'),
              s0=widgets.BoundedFloatText(value=100, min=1, max=1e4, description='$q_0$ / kPa'),
              teval=widgets.BoundedFloatText(value=10, min=0.1, max=100, description='$t_\\mathrm{s}$ / m'),
              n=widgets.BoundedFloatText(value=0.3, min=0, max=0.49, description='$\\nu$'),
              gamma=widgets.BoundedFloatText(value=18, min=16, max=24, description='$\\gamma$ / kPa m$^{-1}$')
    )
    def plot_MC_states(a,b,s0,teval,n,gamma):
        t = np.linspace(1,100,1000) # m
        sns.set_palette("tab10")
        fig, ax = plt.subplots(figsize=(12,8))
        s_ratio = lambda ts: u3D_Rechteck(ts,a,b,s0=s0,nu=n)/u3D_inf(a,b,s0=s0,nu=n)*100
        sig_ratio = lambda ts: i_Rechteck(ts,a,b)*s0 / (gamma * ts) *100
        ax.plot(t,s_ratio(t),label='settlement ratio')
        ax.plot(t,sig_ratio(t),label='stress ratio')
        ax.plot(teval,s_ratio(teval),marker='o',color='red')
        ax.plot(teval,sig_ratio(teval),marker='o',color='red')
        ax.plot([teval,teval],[0,np.maximum(s_ratio(teval),s_ratio(teval))],ls='--',color='black')
        ax.plot([teval,0],[s_ratio(teval),s_ratio(teval)],ls='--',color='black')
        ax.plot([teval,0],[sig_ratio(teval),sig_ratio(teval)],ls='--',color='black')
        ax.text(teval,s_ratio(teval)+5,"%.1f" %s_ratio(teval))
        ax.text(teval,sig_ratio(teval)+5,"%.1f" %sig_ratio(teval))
        limit_f = lambda z: u3D_Rechteck(z,a,b,nu=n)/u3D_inf(a,b,nu=n) - 0.95
        res = scp.optimize.root(fun=limit_f,x0=10)
        ax.set_xlabel('$t$ / m')
        ax.set_ylabel('ratio / %')
        ax.set_xlim(left=0,right=res.x)
        ax.set_ylim(bottom=0)
        ax.legend()
        fig.tight_layout();

In [2]:
def fR(t,a,b,nu=0):
    Rad = np.sqrt(a**2+b**2+t**2)
    fak1 = (1-nu**2)/(b*np.pi)
    fak2 = (1-nu-2*nu**2)/(2*b*np.pi)
    t1 = a*np.log( (b + np.sqrt(a**2+b**2)) * np.sqrt(a**2+t**2) / (a*(b+Rad)) )
    t2 = b*np.log( (a + np.sqrt(a**2+b**2)) * np.sqrt(b**2+t**2) / (b*(a+Rad)) )
    t3 = t * np.arctan(a*b/(t*Rad))
    return fak1 * (t1 + t2) + fak2 * t3

In [2]:
def fK(t,a,b,nu=0):
    ai = [0.87*a,0.87*a,0.87*b,0.13*a]
    bi = [0.87*b,0.13*b,0.13*a,0.13*b]
    f = 0
    for i in range(4):
        f += fR(t,ai[i],bi[i],nu)*bi[i]/b
    return f

In [ ]:
def interactive_fR_rechteck():
    @interact(#c=widgets.BoundedFloatText(value=0.3,min=0, description='$c$ / MPa'),
               z_to_b=widgets.BoundedFloatText(value=1, min=0.1, max=100., description='$z$ / $b$'),
               a_to_b=widgets.BoundedFloatText(value=2, min=0.1, max=100., description='$a$ / $b$')
    )
    def plot_MC_states(z_to_b,a_to_b):
        plt.rcParams['axes.spines.right'] = True
        fig, ax = plt.subplots(figsize=(10,10))
        sns.set_palette("Paired",n_colors=12)
        ax2 = ax.twiny()
        z1 = np.logspace(-1,2,100)
        z2 = np.logspace(-1,0,100)
        for a in [0.1,0.2,0.4,0.8,1,1.5,2,3,5,10,100]:
            ax.plot(fR(z1,a,1),-z1,label='$%.1f$' %a)
            ax2.plot(fR(z2,a,1),-z2)
        
        f = fR(z_to_b,a_to_b,1)
        ax.set_xlim(left=0)
        ax.set_ylim(-100,-0.1)
        ax2.set_xlim(left=0)
        ax.grid(which='both')
        ax.set_yscale('symlog',linthresh=0.01)
        ax.set_yticks([-0.1,-1,-10,-100])
        ax.set_yticklabels([0.1,1,10,100])
        ax.set_yticks(-np.logspace(-1,2,10), minor=True)
        ax.set_xlabel('$f_\\mathrm{R}$')
        ax2.set_xlabel('$f_\\mathrm{R}$')
        ax.set_ylabel('$z$ / $b$')

        if (z_to_b < 1):
            ax2.plot(fR(z2,a_to_b,1),-z2,label='$a$ / $b = %.1f$' %a_to_b,color='black',ls='--')
            ax2.plot([0.,f,f],[-z_to_b,-z_to_b,-0.1],color='black',ls=':')
            ax2.plot(f,-z_to_b,marker='o',color='red')
        else: 
            ax.plot(fR(z1,a_to_b,1),-z1,label='$a$ / $b = %.1f$' %a_to_b,color='black',ls='--')
            ax.plot([0,f,f],[-z_to_b,-z_to_b,-100],color='black',ls=':')
            ax.plot(f,-z_to_b,marker='o',color='red')
        ax.text(0.8,-2,'$s = %.6f q_0 b/E^*$' %f)
        ax2.spines['bottom'].set_position(('data', -0.1))
        ax2.xaxis.set_label_position('top') 
        ax2.xaxis.tick_top()
        fig.legend(bbox_to_anchor=(1.15, 0.8))
        #fig.tight_layout()
        plt.show();

In [ ]:
def interactive_fR_rechteck_nu():
    @interact(z_to_b=widgets.BoundedFloatText(value=1, min=0.1, max=100., description='$z$ / $b$'),
              a_to_b=widgets.BoundedFloatText(value=2, min=0.1, max=100., description='$a$ / $b$'),
              nu=widgets.BoundedFloatText(value=0.3, min=0, max=0.49, description='$\\nu$')
    )
    def plot_MC_states(z_to_b,a_to_b,nu):
        plt.rcParams['axes.spines.right'] = True
        fig, ax = plt.subplots(figsize=(10,10))
        sns.set_palette("Paired",n_colors=12)
        z1 = np.logspace(-1,2,100)
        for a in [0.1,0.2,0.4,0.8,1,1.5,2,3,5,10,100]:
            ax.plot(fR(z1,a,1,nu=nu),-z1,label='$%.1f$' %a)
            ax.plot(fR(z1,a,1),-z1,ls='--')
        
        f = fR(z_to_b,a_to_b,1,nu=nu)
        ax.set_xlim(left=0)
        ax.set_ylim(-100,-0.1)
        ax.grid(which='both')
        ax.set_yscale('symlog',linthresh=0.01)
        ax.set_yticks([-0.1,-1,-10,-100])
        ax.set_yticklabels([0.1,1,10,100])
        ax.set_yticks(-np.logspace(-1,2,10), minor=True)
        ax.set_xlabel('$f_\\mathrm{R}$')
        ax.set_ylabel('$z$ / $b$')

        ax.plot(fR(z1,a_to_b,1,nu=nu),-z1,label='$a$ / $b = %.1f$' %a_to_b,color='black')
        ax.plot([0,f,f],[-z_to_b,-z_to_b,-100],color='black',ls=':')
        ax.plot(f,-z_to_b,marker='o',color='red')
        ax.text(0.8,-2,'$s = %.6f q_0 b/E$' %f)
        fig.legend(bbox_to_anchor=(1.15, 0.8))
        #fig.tight_layout()
        ax.plot(0,-0.1,color='black',ls='-',label='$\\nu = %.1f$' %nu)
        ax.plot(0,-0.1,color='black',ls='--',label='$\\nu = 0$')
        plt.show();

In [ ]:
def interactive_fK_rechteck_nu():
    @interact(z_to_b=widgets.BoundedFloatText(value=1, min=0.1, max=100., description='$z$ / $b$'),
              a_to_b=widgets.BoundedFloatText(value=2, min=0.1, max=100., description='$a$ / $b$')
    )
    def plot_MC_states(z_to_b,a_to_b):
        plt.rcParams['axes.spines.right'] = True
        fig, ax = plt.subplots(figsize=(10,10))
        sns.set_palette("Paired",n_colors=12)        
        ax2 = ax.twiny()
        z1 = np.logspace(-1,2,100)
        z2 = np.logspace(-1,0,100)
        for a in [0.1,0.2,0.4,0.8,1,1.5,2,3,5,10,100]:
            ax.plot(fK(z1,a,1),-z1,label='$%.1f$' %a)
            ax2.plot(fK(z2,a,1),-z2)
        
        f = fK(z_to_b,a_to_b,1)
        ax.set_xlim(left=0)
        ax.set_ylim(-100,-0.1)
        ax2.set_xlim(left=0)
        ax.grid(which='both')
        ax.set_yscale('symlog',linthresh=0.01)
        ax.set_yticks([-0.1,-1,-10,-100])
        ax.set_yticklabels([0.1,1,10,100])
        ax.set_yticks(-np.logspace(-1,2,10), minor=True)
        ax.set_xlabel('$f_\\mathrm{K}$')
        ax2.set_xlabel('$f_\\mathrm{K}$')
        ax.set_ylabel('$z$ / $b$')

        if (z_to_b < 1):
            ax2.plot(fK(z2,a_to_b,1),-z2,label='$a$ / $b = %.1f$' %a_to_b,color='black',ls='--')
            ax2.plot([0.,f,f],[-z_to_b,-z_to_b,-0.1],color='black',ls=':')
            ax2.plot(f,-z_to_b,marker='o',color='red')
        else: 
            ax.plot(fK(z1,a_to_b,1),-z1,label='$a$ / $b = %.1f$' %a_to_b,color='black',ls='--')
            ax.plot([0,f,f],[-z_to_b,-z_to_b,-100],color='black',ls=':')
            ax.plot(f,-z_to_b,marker='o',color='red')
        ax.text(2,-2,'$s_\\mathrm{K} = %.6f q_0 b/E^*$' %f)
        ax2.spines['bottom'].set_position(('data', -0.1))
        ax2.xaxis.set_label_position('top') 
        ax2.xaxis.tick_top()
        fig.legend(bbox_to_anchor=(1.15, 0.8))
        #fig.tight_layout()
        plt.show();

In [4]:
def run_layers(update_layers):
    global items
    items = [widgets.BoundedFloatText(description='$q_0$ / kPa',value=300,min=0,max=2000),
         widgets.BoundedFloatText(description='$d$ / m',value=2,min=0,max=100),
         widgets.BoundedFloatText(description='$\\gamma_d$ / kN m$^{-3}$',value=18,min=0,max=30),
         widgets.BoundedFloatText(description='$\\nu$',value=0,min=0,max=0.499),
         widgets.BoundedFloatText(description='$a$ / m',value=2,min=0.01,max=50),
         widgets.BoundedFloatText(description='$b$ / m',value=2,min=0.01,max=50)]
    for i in range(n_layers.value):
        items.append(widgets.BoundedFloatText(description='$z_{%i}$ / m' %(i+1),value=1+i*2,min=0,max=200))
        items.append(widgets.BoundedFloatText(description='$E^*_{%i}$ / MPa' %(i+1),value=(i+1)*10,min=0.01,max=10000))
        items.append(widgets.BoundedFloatText(description='$\gamma_{%i}$ / kN m$^{-3}$' %(i+1),value=18-i,min=1,max=35))
    #return None
    return widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(3,350px)"));

In [1]:
def run_analysis(update_analysis):
    global items
    global n_layers
    q0 = items[0].value
    d = items[1].value
    gamma_d = items[2].value
    q1 = q0 - d*gamma_d
    nu = items[3].value
    a = items[4].value
    b = items[5].value
    z0 = items[6].value
    E0 = items[7].value
    gamma0 = items[8].value
    sR = fR(z0,a,b,nu)/E0/1e3
    sK = fK(z0,a,b,nu)/E0/1e3
    zs = np.array([0,z0])
    Es = np.array([E0,E0])
    gammas = np.array([gamma0,gamma0])
    sig0 = np.array([d*gamma_d,d*gamma_d+gamma0*z0])
    dsigR = sigma_z_rectangle2(a/2,b/2,z0,q1,a,b)
    dsigK = sigma_z_rectangle2(a*0.37,0.37*b,z0,q1,a,b)
    delta_sigR = np.array([q1,dsigR])
    delta_sigK = np.array([q1,dsigK])
    sRs = np.array([sR,sR])
    sKs = np.array([sK,sK])
    for i in range(1,n_layers.value):
        sRs = np.append(sRs,(fR(items[6+i*3].value,a,b,nu) - fR(items[6+(i-1)*3].value,a,b,nu))/items[7+i*3].value/1e3)
        sKs = np.append(sKs,(fK(items[6+i*3].value,a,b,nu) - fK(items[6+(i-1)*3].value,a,b,nu))/items[7+i*3].value/1e3)
        zs = np.append(zs,items[6+i*3].value)
        Es = np.append(Es,items[7+i*3].value)
        gammas = np.append(gammas,items[8+i*3].value)
        sig0 = np.append(sig0,sig0[-1]+items[8+i*3].value*(items[6+i*3].value - items[6+(i-1)*3].value))
        delta_sigR = np.append(delta_sigR,sigma_z_rectangle2(a/2,b/2,items[6+i*3].value,q1,a,b))
        delta_sigK = np.append(delta_sigK,sigma_z_rectangle2(a*0.37,0.37*b,items[6+i*3].value,q1,a,b))
    
    #
    sRs *= q1*b
    sKs *= q1*b
    total_sRs = np.append(np.flip(np.flip(sRs[1:]).cumsum()),0)
    total_sKs = np.append(np.flip(np.flip(sKs[1:]).cumsum()),0)
    print("Die wirksame Sohlpressung beträgt %.2f kN/m²" %q1)
    print("Die Setzung am Eckpunkt beträgt %.2f cm" %(sRs[1:].sum()*100))
    print("Die Setzung am kennzeichnenden Punkt beträgt %.2f cm" %(sKs[1:].sum()*100))
    #
    plt.rcParams['axes.spines.right'] = False
    sns.set_palette("tab10")
    fig, ax = plt.subplots(ncols=4,figsize=(20,8))
    ax[0].step(Es,zs,where='post')
    ax[0].invert_yaxis()
    for i in range(0,n_layers.value,2):
        ax[0].axhspan(zs[i],zs[i+1],alpha=0.1,color='gray')
    for i in range(n_layers.value):
        ax[0].text(np.min(Es),(zs[i]+zs[i+1])/2,'layer %i' %(i+1))
    ax[0].set_xlabel('$E^*$ / MPa')
    ax[0].set_ylabel('$z$ / m')
    #
    ax[1].step(gammas,zs,where='post')
    ax[1].invert_yaxis()
    for i in range(0,n_layers.value,2):
        ax[1].axhspan(zs[i],zs[i+1],alpha=0.1,color='gray')
    for i in range(n_layers.value):
        ax[1].text(np.min(gammas),(zs[i]+zs[i+1])/2,'layer %i' %(i+1))
    ax[1].set_xlabel('$\\gamma$ / kN m$^{-3}$')
    ax[1].set_ylabel('$z$ / m')
    #
    ax[2].plot(sig0,zs,label='$\\sigma_0$')
    ax[2].plot(delta_sigR,zs,label='$\\Delta \\sigma_\\mathrm{E}$')
    ax[2].plot(delta_sigK,zs,label='$\\Delta \\sigma_\\mathrm{K}$')
    ax[2].invert_yaxis()
    for i in range(0,n_layers.value,2):
        ax[2].axhspan(zs[i],zs[i+1],alpha=0.1,color='gray')
    for i in range(n_layers.value):
        ax[2].text(np.min(sig0),(zs[i]+zs[i+1])/2,'layer %i' %(i+1))
    ax[2].set_xlabel('$\\sigma$ / kN m$^{-2}$')
    ax[2].set_ylabel('$z$ / m')
    ax[2].legend()
    #
    ax[3].step(sRs*1e2,zs,label='$\\Delta s_\\mathrm{E}$',where='post',ls='--')
    ax[3].step(sKs*1e2,zs,label='$\\Delta s_\\mathrm{K}$',where='post',ls='--')
    ax[3].plot(total_sRs*1e2,zs,label='$s_\\mathrm{E}$')
    ax[3].plot(total_sKs*1e2,zs,label='$s_\\mathrm{K}$')
    ax[3].invert_yaxis()
    for i in range(0,n_layers.value,2):
        ax[3].axhspan(zs[i],zs[i+1],alpha=0.1,color='gray')
    for i in range(n_layers.value):
        ax[3].text(np.min(sRs),(zs[i]+zs[i+1])/2,'layer %i' %(i+1))
    ax[3].set_xlabel('$s$ / cm')
    ax[3].set_ylabel('$z$ / m')
    ax[3].legend()
    fig.tight_layout()
    plt.show();

In [1]:
def get_secant(s0,ds):
    eps = lambda sig, Eref, sigref, om: sigref/((1-om)*Eref) * ( np.power(sig/sigref,1-om) -1 )
    eps0 = eps(s0, 1000, 10, 0.9)
    eps1 = eps(s0+ds, 1000, 10, 0.9)
    #return s0/s0*20e3
    return ds/(eps1 - eps0)

In [ ]:
def interactive_secant():
    @interact(
        s0 = widgets.BoundedFloatText(value=100,min=10,max=1000,description='$\\sigma_0$ / kPa'),
        ds = widgets.BoundedFloatText(value=300,min=1,max=1000,description='$\\Delta \\sigma$ / kPa')        
    )
    def plot_secant(s0, ds):
        eps = lambda sig, Eref, sigref, om: sigref/((1-om)*Eref) * ( np.power(sig/sigref,1-om) -1 )
        sigma = np.linspace(10,1000,1000) #kPa
        eps0 = eps(s0, 1000, 10, 0.9)
        eps1 = eps(s0+ds, 1000, 10, 0.9)
        E_sec = ds/(eps1 - eps0)
        fig, ax = plt.subplots()
        ax.plot(sigma,eps(sigma,1000,10,0.9)*100)
        ax.plot([s0,s0+ds],[eps0*100,eps1*100],marker='o',color='red')
        ax.text(400,1,'$E_\\mathrm{s}^\\mathrm{sec} = %.2f$ MPa' %(E_sec/1e3))
        ax.set_xlabel("$\\sigma'$ / kPa")
        ax.set_ylabel('$\epsilon$ / %')
        ax.invert_yaxis()
        fig.tight_layout()
        plt.show();

In [2]:
def plot_stiffness_input():
    sigma = np.linspace(10,1000,1000) #kPa

    fig, ax = plt.subplots(ncols=3,figsize=(18,8))
    ax[0].plot(sigma,eps(sigma,1000,10,0.9)*100)
    ax[0].set_xlabel("$\\sigma'$ / kPa")
    ax[0].set_ylabel('$\epsilon$ / %')
    ax[0].invert_yaxis()

    ax[1].plot(sigma,eps(sigma,1000,10,0.9)*100)
    ax[1].set_xlabel("$\\sigma'$ / kPa")
    ax[1].set_ylabel('$\epsilon$ / %')
    ax[1].invert_yaxis()
    ax[1].set_xscale('log')

    ax[2].plot(sigma,E_s(sigma,1000,10,0.9)/1e3)
    ax[2].set_xlabel("$\\sigma'$ / kPa")
    ax[2].set_ylabel('$E_\\mathrm{s}$ / MPa')
    fig.tight_layout();

In [3]:
def plot_secant_tangent():
    fig, ax = plt.subplots(figsize=(12,8))
    z = np.linspace(10/gamma,ts,100)
    Esec = get_secant(np.maximum(gamma*z,0),q0*i_K_Rechteck(z,a,b))
    ax.plot(Esec/1e3,z,label='Sekantenmodul')
    ax.plot(E_s(np.maximum(gamma*z,0),1000,10,0.9)/1e3,z, label='Tangentenmodul (bei $\\sigma_0$)',ls='--')
    ax.plot(E_s(np.maximum(gamma*z,0)+q0*i_K_Rechteck(z,a,b),1000,10,0.9)/1e3,z, label='Tangentenmodul (bei $\\sigma_0+\\Delta \\sigma$)',ls='--')
    ax.plot(E_s(np.maximum(gamma*z,0)+0.5*q0*i_K_Rechteck(z,a,b),1000,10,0.9)/1e3,z, label='Tangentenmodul (bei $\\sigma_0+\\Delta \\sigma/2$)',ls='--')
    #ax.plot(E_s(np.maximum(gamma*z,0)+0.35*q0*i_K_Rechteck(z,a,b),1000,10,0.9)/1e3,z, label='Tangentenmodul (bei $\\sigma_0+ 0.35 \\Delta \\sigma$)',ls='--')
    ax.invert_yaxis()
    ax.set_xlabel('$E_\\mathrm{s}$ / MPa')
    ax.set_ylabel('$z$ / m')
    ax.legend()
    fig.tight_layout();

In [ ]:
def interactive_direct_layers():
    @interact(
        nlayers = widgets.IntSlider(value=3,min=1,max=15,description='$n$'),
        d = widgets.FloatSlider(value=0,min=0,max=3,description='$d$ / m')
    )
    def run_layers_direct(nlayers,d):
        nlayers += 1
        global gamma, q0, a, b, ts
        zu = np.linspace(0,ts,nlayers)
        zm = np.array([zu[i-1]+(zu[i] - zu[i-1])/2 for i in range(1,nlayers)])

        problem = pd.DataFrame(np.array([zu[1:],
                                          zm]).T,
                           columns=['Schichtunterkante / m', 'Schichtmitte / m'])

        s0 = gamma * (zm+d)
        ds = (q0-gamma*d)*i_K_Rechteck(zm,a,b)
        Esec = get_secant(s0,ds)
        fKs = fK(zu[1:],a,b,0)

        du = np.array([fKs[0]/Esec[0]])
        for i in range(1,nlayers-1):
            du = np.append(du,(fKs[i]-fKs[i-1])/Esec[i])
        du *= q0*b

        problem['Ausgangsspannung / kPa'] = s0
        problem['Spannungszuwachs / kPa'] = ds
        problem['Spannungsverhältnis / %'] = ds/s0*100
        problem['Rechenmodul / MPa'] = Esec/1e3
        problem['Setzungsbeiwerte'] = fKs
        problem['Setzungsbeiträge / cm'] = du*100
        display(problem)
        first_index_below_20 = problem[problem['Spannungsverhältnis / %'].lt(20)].index[0]
        print("Die Gesamtsetzung bei Zugrundelegung von ts beträgt %.2f cm" %(du.sum()*100))
        print("Die Gesamtsetzung bei Zugrundelegung der 20-Prozent-Regel beträgt %.2f cm (Schichten 0 bis %i)" %(du[0:first_index_below_20+1].sum()*100,first_index_below_20))

In [1]:
def interactive_moduli_layers():
    @interact(
        nlayers = widgets.IntSlider(value=3,min=1,max=15,description='$n$'),
        d = widgets.FloatSlider(value=0,min=0,max=3,description='$d$ / m')
    )
    def run_layers_direct(nlayers,d):
        nlayers += 1
        global gamma, q0, a, b, ts
        zu = np.linspace(0,ts,nlayers)
        zm = np.array([zu[i-1]+(zu[i] - zu[i-1])/2 for i in range(1,nlayers)])
        dz = np.array([(zu[i] - zu[i-1]) for i in range(1,nlayers)])

        problem = pd.DataFrame(np.array([zu[1:],
                                          zm]).T,
                           columns=['Schichtunterkante / m', 'Schichtmitte / m'])

        s0 = gamma * (zm+d)
        ds = (q0-gamma*d)*i_K_Rechteck(zm,a,b)
        Esec = get_secant(s0,ds)

        du = ds/Esec*dz
        
        problem['Ausgangsspannung / kPa'] = s0
        problem['Spannungszuwachs / kPa'] = ds
        problem['Sekantenmodul / MPa'] = Esec/1e3
        problem['Tangentenmodul / MPa'] = E_s(s0+ds/2,1000,10,0.9)/1e3
        display(problem)

In [1]:
def interactive_indirect_layers():
    @interact(
        nlayers = widgets.IntSlider(value=3,min=1,max=15,description='$n$'),
        d = widgets.FloatSlider(value=0,min=0,max=3,description='$d$ / m')
    )
    def run_layers_direct(nlayers,d):
        nlayers += 1
        global gamma, q0, a, b, ts
        zu = np.linspace(0,ts,nlayers)
        zm = np.array([zu[i-1]+(zu[i] - zu[i-1])/2 for i in range(1,nlayers)])
        dz = np.array([(zu[i] - zu[i-1]) for i in range(1,nlayers)])

        problem = pd.DataFrame(np.array([zu[1:],
                                          zm]).T,
                           columns=['Schichtunterkante / m', 'Schichtmitte / m'])

        s0 = gamma * (zm+d)
        ds = (q0-gamma*d)*i_K_Rechteck(zm,a,b)
        Esec = get_secant(s0,ds)

        du = ds/Esec*dz
        
        problem['Schichtmächtigkeit / m'] = dz
        problem['Ausgangsspannung / kPa'] = s0
        problem['Spannungszuwachs / kPa'] = ds
        problem['Spannungsverhältnis / %'] = ds/s0*100
        problem['Rechenmodul / MPa'] = Esec/1e3
        problem['Setzungsbeiträge / cm'] = du*100
        display(problem)
        first_index_below_20 = problem[problem['Spannungsverhältnis / %'].lt(20)].index[0]
        print("Die Gesamtsetzung bei Zugrundelegung von ts beträgt %.2f cm" %(du.sum()*100))
        print("Die Gesamtsetzung bei Zugrundelegung der 20-Prozent-Regel beträgt %.2f cm (Schichten 0 bis %i)" %(du[0:first_index_below_20+1].sum()*100,first_index_below_20))

In [1]:
def plot_aushub():
    fig, ax = plt.subplots(figsize=(22,6),ncols=5)
    #original re-consolidation
    range0 = np.logspace(0,0.5,100)
    #virgin compression
    range1 = np.logspace(0.5,1.8,100)
    eps0_end = eps(range0[-1],1e3,1,0.9)
    eps1_beg = eps(range1[0],1e2,1,0.9)
    #unloading and reloading
    range2 = np.logspace(0.7,1.8,100)
    weight = (np.log(range2) - np.log(range2[0]))/(np.log(range2[-1]) - np.log(range2[0]))
    eps1_end = eps(range2[-1],1e2,1,0.9)+eps0_end-eps1_beg
    eps2_beg = eps(range2[-1],5e2,1,0.9)
    #virgin compression
    range3 = np.logspace(1.8,3,100)

    for i in range(5):
        ax[i].plot(range0,eps(range0,1e3,1,0.9),color='black')
        ax[i].plot(range1,eps(range1,1e2,1,0.9)+eps0_end-eps1_beg,color='black')

    for i in range(1,5):
        ax[i].plot(range2,eps(range2,5e2,1,0.9)+eps1_end-eps2_beg+loopifier(weight),color='red')

    for i in range(2,5):
        ax[i].plot(range2,eps(range2,5e2,1,0.9)+eps1_end-eps2_beg-loopifier(weight),color='green')

    for i in range(3,5):
        ax[i].plot(range3,eps(range3,1e2,1,0.9)+eps0_end-eps1_beg,color='blue')

    ax[0].set_title('Vor Aushub')
    ax[0].plot(range1[-1],eps1_end,marker='o',markersize='12',color='orange')

    ax[1].set_title('Nach Aushub')
    ax[1].plot(range2[0],eps(range2[0],5e2,1,0.9)+eps1_end-eps2_beg,marker='o',markersize='12',color='orange')

    ax[2].set_title('Fiktive Auffüllung')
    ax[2].plot(range2[-1],eps(range2[-1],5e2,1,0.9)+eps1_end-eps2_beg,marker='o',markersize='12',color='orange')

    ax[3].set_title('Nach Lastaufbringung')
    ax[3].plot(range3[-1],eps(range3[-1],1e2,1,0.9)+eps0_end-eps1_beg,marker='o',markersize='12',color='orange')

    ax[4].set_title('Sekantensteifemodul')
    ax[4].plot(range2[-1],eps(range2[-1],5e2,1,0.9)+eps1_end-eps2_beg,marker='o',markersize='12',color='orange')
    ax[4].plot(range3[-1],eps(range3[-1],1e2,1,0.9)+eps0_end-eps1_beg,marker='o',markersize='12',color='orange')
    ax[4].plot([range2[-1],range3[-1]],[eps(range2[-1],5e2,1,0.9)+eps1_end-eps2_beg,eps(range3[-1],1e2,1,0.9)+eps0_end-eps1_beg],ls='--',color='orange')


    for i in range(5):
        ax[i].invert_yaxis()
        if i < 4:
            ax[i].set_xscale('log')
        ax[i].set_xticklabels([])
        ax[i].set_yticklabels([])
        ax[i].set_xlabel('$\\sigma$')
        ax[i].set_ylabel('$\\epsilon$')
    fig.suptitle('Einfluss der inkrementellen Nichtlinearität des Kompressionsverhaltens von Böden',fontweight='bold')
    fig.tight_layout();

In [ ]:
def stress_depth(dzw):
    zw0=2
    gamma=16
    gamma_s=21
    plt.rcParams['axes.spines.top'] = True
    z = np.linspace(0,15,1000)
    pw0 = lambda dzw: np.maximum(z*10-(zw0+dzw)*10,0)
    gamma_tot = lambda dzw: gamma*(1-np.heaviside(z-(zw0+dzw),0)) + gamma_s*np.heaviside(z-(zw0+dzw),0)
    s0 = lambda dzw: z*gamma_tot(dzw) - (gamma_s - gamma) * (zw0+dzw) * np.heaviside(z-(zw0+dzw),0)
    sp0 = lambda dzw: s0(dzw) - pw0(dzw)
    E_s = lambda sig, Eref, sigref, om: Eref * np.maximum(np.power(sig/sigref,om),1)
    #
    fig, ax = plt.subplots(ncols=4,figsize=(18,8))
    ax[0].plot(pw0(0),z,label='$u$')
    ax[0].plot(s0(0),z,label='$\\sigma_{zz}$')
    ax[0].plot(sp0(0),z,label="$\\sigma'_{zz}$")
    ax[0].axhline(zw0,ls='--',color='blue',label='GW')
    #
    ax[1].plot(pw0(dzw),z,label='$u$')
    ax[1].plot(s0(dzw),z,label='$\\sigma_{zz}$')
    ax[1].plot(sp0(dzw),z,label="$\\sigma'_{zz}$")
    ax[1].axhline(zw0+dzw,ls='--',color='blue',label='GW')
    #
    ax[2].plot(pw0(dzw)-pw0(0),z,label='$\\Delta u$')
    ax[2].plot(s0(dzw)-s0(0),z,label='$\\Delta \\sigma_{zz}$')
    ax[2].plot(sp0(dzw)-sp0(0),z,label="$\\Delta \\sigma'_{zz}$")
    ax[2].axhline(zw0,ls=':',color='blue',label='GW1')
    ax[2].axhline(zw0+dzw,ls='--',color='blue',label='GW2')
    #
    eps = lambda dzw: (sp0(dzw)-sp0(0))/(E_s((sp0(dzw)+sp0(0))/2,1e4,10,0.9))
    du = lambda dzw: eps(dzw) * (z[1]-z[0])
    u = lambda dzw: np.flip(np.flip(du(dzw)).cumsum())
    ax[3].plot(eps(dzw)*100,z,label='$\\epsilon_{zz}$')
    ax2 = ax[3].twiny()
    ax2.plot(u(dzw)*100,z,label='$u_z(t_\\mathrm{s} = 15\,\\mathrm{m})$',color='r')
    ax2.set_xlabel('$u_z$ / cm')
    ax2.legend(loc=4)
    for i in range(4):
        ax[i].legend()
        ax[i].set_ylabel('$z$ / m')
        ax[i].set_xlabel('$\\sigma, u$ / kPa')
        if i < 3:
            ax[i].spines['top'].set_linewidth(0)
        if i == 3:
            ax[i].set_xlabel('$\epsilon$ / %')
        ax[i].invert_yaxis()
    fig.tight_layout()
    plt.show();